In [4]:
import numpy as np
import matplotlib.pyplot as pyplot
import random

In [110]:
# utils function

def compute_magnitude(pts):
    """Return a vector of magnitudes of an array of points in R^2"""
    return np.sqrt(np.power(pts[:, 0], 2) + np.power(pts[:, 1], 2))   

def compute_center_mass(pts):
    """Return the center of mass of points in R^2"""
    return np.array([pts.mean(axis=1)])

def compute_dispersion(pts, center_mass):
    """Return the dispersion of the points as the average distance from the center of mass"""
    return compute_magnitude(pts - center_mass).mean()

In [111]:
class Swarm:
    """Class swarm that implementes PSO for 2D problems"""

    def __init__(
        self,
        n,
        fit,
        soc_factor=1.49,
        cog_factor=1.49,
        inertia_weight=0.9,
        vel_magnitude_limits=[0.01, 50],
        ):

        self.n = n # number of particles
        self.fit = fit # fitness function

        self.soc_factor = soc_factor
        self.cog_factor = cog_factor
        self.inertia_weight = inertia_weight
        self.inertia_decay = 0.99
        self.inertia_min = 0

        self.vel_magnitude_limits = vel_magnitude_limits # min and max velocity magnitude

        # initialize
        self.swarm = None
        self.velocity = None
        self.initialization()

        self.personal_best = self.swarm
        self.global_best = np.array([self.best[np.argmin(self.fit(self.best))]])
        self.center_mass = compute_center_mass(self.swarm)
        self.dispersion = compute_dispersion(self.swarm, self.center_mass) # average distance from center of mass

    def initialization(self, gen_position_limits=np.array([[-5, 5]]*2)):
        
        """Function dealing with the initialization of the particles initial positions and velocities"""

        # initialize population
        self.swarm = np.random.uniform(
            gen_position_limits[:, 0], 
            gen_position_limits[:, 1], 
            size=(self.n, 2))
        
        # initialize velocity
        self.velocity = np.zeros((self.n, 2))

    def update_velocity(self):
        # factors
        inertia = self.velocity * self.inertia_weight
        social = self.soc_factor * r1 * (self.global_best - self.swarm)
        cognitive = self.cog_factor * r2 * (self.best - self.swarm)

        # stochastic elements
        r1 = np.random.uniform(np.zeros(2), np.ones(2), size=(self.n, 2))
        r2 = np.random.uniform(np.zeros(2), np.ones(2), size=(self.n, 2))

        # update velocity
        self.velocity = inertia + social + cognitive
        self.inertia_weight = self.inertia_weight * self.inertia_decay
        if self.inertia_weight < self.inertia_min:
            self.inertia_weight = self.inertia_min

        # check velocity lower/upper magnitude limit
        magnitude = compute_magnitude(self.velocity)
        large_vel = magnitude > self.vel_magnitude[1]
        small_vel = magnitude < self.vel_magnitude[0]
        self.velocity[large_vel] = self.velocity[large_vel] / np.array([[magnitude[large_vel]]]).reshape(self.n, 1) * self.vel_magnitude[1]
        self.velocity[large_vel] = self.velocity[small_vel] / np.array([[magnitude[small_vel]]]).reshape(self.n, 1) * self.vel_magnitude[0]

    def update_position(self):
        self.swarm = self.swarm + self.velocity

        # update best
        indexes = self.fit(self.personal_best) > self.fit(self.swarm)
        self.personal_best[indexes] = self.swarm[indexes]
        self.global_best = np.array([self.personal_best[np.argmin(self.fit(self.personal_best))]])

        # update center mass
        self.center_mass = compute_center_mass(self.swarm)

        # compute dispersion
        self.dispersion = compute_dispersion(self.swarm, self.center_mass)

    def generate(self, cycles=1):
        for _ in range(cycles):
            self.update_velocity()
            self.update_position()